# Triggers

A trigger in SQL is a predefined, automatic action or set of actions that are executed in response to a specific event on a database table. These events can be data modification operations, such as **INSERT, UPDATE, DELETE,** or even certain types of system events. Triggers are a part of DML.

Triggers are typically defined using SQL statements and are attached to specific tables. When the specified triggering event occurs, the trigger's associated actions are executed. These actions can include executing SQL statements, invoking stored procedures, or performing other tasks, depending on the logic defined within the trigger.

For your interest, I am not a big fan of triggers. They are automated actions which can be a good and a bad thing at the same time. I do believe that triggers are useful for very common, small impact things, like for example, in Bookshop Near Me database, trigger is activadet whenever specific amount of loyalty points is reached and from there a membership status must be changed for the corresponding customer. I personally prefer working with stored procedures.

## LOYALTY POINTS table

Create a trigger that sums loyalty points to a specific customer (if it exists) and sets the transaction date to 
current date (the date when the transaction is made), if the customer is new (does not exist already in my loyaltypoints
table) then create a new register with the new customer and start summing its loyalty points.


**Trigger to update existing membership status for customer**
```SQL
-- Create a trigger to update membership status
CREATE TRIGGER update_membership_status
AFTER UPDATE ON loyaltypoints
FOR EACH ROW
BEGIN
    DECLARE loyalty_points INT;

    -- Get the loyalty points earned by the customer
    SELECT points_earned INTO loyalty_points
    FROM loyaltypoints
    WHERE customer_id = NEW.customer_id;

    IF loyalty_points >= 1 AND loyalty_points < 100 THEN
        -- Update the membership status in the customers table
        UPDATE customers
        SET membership_status = 'Standard'
        WHERE customer_id = NEW.customer_id;
    ELSEIF loyalty_points >= 100 AND loyalty_points < 500 THEN
        UPDATE customers
        SET membership_status = 'Premium'
        WHERE customer_id = NEW.customer_id;
    ELSEIF loyalty_points >= 500 THEN
        UPDATE customers
        SET membership_status = 'VIP'
        WHERE customer_id = NEW.customer_id;
    END IF;
END
//

DELIMITER ;


**Insert a newly gained membership status for customer**
```sql
DELIMITER //
CREATE TRIGGER insert_membership_status
AFTER INSERT ON loyaltypoints
FOR EACH ROW
BEGIN
    DECLARE earnedPoints INT;
    DECLARE customerMembershipStatus VARCHAR(20);

    -- Get the loyalty points earned by the customer
    SELECT NEW.points_earned INTO earnedPoints;

    -- Determine the new membership status based on loyalty points
    CASE
        WHEN earnedPoints >= 1 AND earnedPoints < 100 THEN
            SET customerMembershipStatus = 'Standard';
        WHEN earnedPoints >= 100 AND earnedPoints < 500 THEN
            SET customerMembershipStatus = 'Premium';
        WHEN earnedPoints >= 500 THEN
            SET customerMembershipStatus = 'VIP';
        ELSE
            SET customerMembershipStatus = NULL; -- Default status if points not in range
    END CASE;

    -- Update the membership status in the customers table
    UPDATE customers
    SET membership_status = customerMembershipStatus
    WHERE customer_id = NEW.customer_id;
END;
//
DELIMITER ;


## REVIEWS table

### Trigger on `AFTER INSERT`

Create a trigger after a review is inserted for specific book
```sql
DELIMITER //
CREATE TRIGGER update_average_rating
AFTER INSERT ON reviews
FOR EACH ROW
BEGIN
    DECLARE total_ratings INT;
    DECLARE total_rating_sum DECIMAL(10, 2);
    DECLARE new_average_rating DECIMAL(3, 1);

    -- Calculate the total number of ratings for the book
    SELECT COUNT(*) INTO total_ratings
    FROM reviews
    WHERE book_id = NEW.book_id;

    -- Calculate the sum of all ratings for the book
    SELECT SUM(LEAST(NEW.rating, 5)) INTO total_rating_sum
    FROM reviews
    WHERE book_id = NEW.book_id;

    -- Calculate the new average rating, ensuring it's between 1 and 5
    IF total_ratings = 0 THEN
        SET new_average_rating = 0;  -- Handle the case when there are no reviews yet.
    ELSE
        SET new_average_rating = LEAST((total_rating_sum / total_ratings), 5);
    END IF;

    -- Update the book table with the new average rating
    UPDATE books
    SET avarage_rating = new_average_rating
    WHERE book_id = NEW.book_id;
END
//

DELIMITER ;


## USERS table

Create a trigger whenever a new user is created - before insert - so it's password is hashed
```sql
DELIMITER //
CREATE TRIGGER hashedPassword
BEFORE INSERT ON users
FOR EACH ROW
BEGIN
    SET NEW.salting = UNHEX(SUBSTRING(SHA2(UUID(),256),1,32));
    SET NEW.password = SHA2(CONCAT(NEW.salting, NEW.password),256);
END;
//
DELIMITER ;


## ORDER ITEMS table

Creating a trigger that whenever a new orderitem is inserted with specific book quantity in it, it is substracted from books table stock_quantity

```sql
DELIMITER //

CREATE TRIGGER update_stock_quantity
AFTER INSERT ON orderitems
FOR EACH ROW
BEGIN
    DECLARE book_stock INT;
    
    -- Get the current stock_quantity of the book
    SELECT stock_quantity INTO book_stock
    FROM books
    WHERE book_id = NEW.book_id;
    
    -- Update the stock_quantity
    UPDATE books
    SET stock_quantity = book_stock - NEW.quantity
    WHERE book_id = NEW.book_id;
END;
//

DELIMITER ;
